# Implementing [Schmid 2019] using SQLite

[Bradley P. Allen](https://www.bradleypallen.org), 2022-05-08

## Overview

This Python notebook gist contains a very simple implementation of a relational database approach to storing and querying labeled property graphs as described by Matthias Schmid of the University of Passau [[1]](https://dl.acm.org/doi/abs/10.1145/3366030.3366046).

## Preliminaries

### Python library imports

First, we import some Python libraries. We'll use ```hashlib``` to give us an stable cryptographic hash for shredding edges into adjacency tables, ```pandas``` to print out query results in an attractive format, and Simon Willison's ```sqlite_utils``` to make the task of manipulating the SQLite database easy.

In [1]:
import hashlib
import pandas as pd
from sqlite_utils import Database

### Demo database cleanup

We'll also delete and recreate the SQLite database file we'll be using, just to keep things simple.

In [2]:
!rm -f test-v0.4.db

In [3]:
db = Database('test-v0.4.db')

### A simple function for hashing of edge labels to column pairs

Schmid's approach, based on previous work ([[2]](https://d1wqtxts1xzle7.cloudfront.net/69945800/Building_an_efficient_RDF_store_over_a_r20210919-15810-1bebrk0-with-cover-page-v2.pdf?Expires=1652064544&Signature=DoStFyoFgEEBo5sRj1nCLaclUvoTXfhywyMcoytJNboAy~1Zf0LQi~SAeS1guJXpsi4QfS2F7Pbso3rK~U-VNPzJxMgY1mC0IgjoqcuXh025cmvKJJi-OTTb1S87tbBaEi0Dd20bf39-jaLdI1u97iDOdj88uUcaOC14m54DgxdMm9nc1FpZzM-DuyP-LWD-eyQ0qoNxZqjnCs48Tjb9RjAy3OgaAdXlEgLukQWJezhVdbkjzuG8Qh~egRpXzAAJurN4pUYk7F6UC~8Ep5IPm0Z1qO~kDCYuJ7MIAGuE-8Z9M1zb-BI3BHF7tgphspndjVhQQLX2R5JW4rjtQvNZfQ__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA), [[3]](https://dl.acm.org/doi/abs/10.1145/2723372.2723732)), uses a graph coloring algorithm to derive a hashing function that avoids collisions in the adjacency tables. We didn't get around to doing that yet. Instead, collisions be damned; we'll just hash the edge label to one of ```NUMBER_OF_COLUMN_PAIRS``` column pairs. 

In [4]:
NUMBER_OF_COLUMN_PAIRS = 100

In [5]:
def column_pair(label):
    m = hashlib.md5()
    m.update(str.encode(label))
    digest = m.hexdigest()
    n = int(digest, 16) % NUMBER_OF_COLUMN_PAIRS
    return ( f'label_{n}', f'edges_{n}' )

### A utility function for generating adjacency tables from edges

Schmid uses column triples in his adjacency table schema, with one column containing an array of edge ids, another column containing an array of target vertex ids, and a column containing the edge label common to all of those. His ```unshred_edges``` Common Table Expression (CTE) then uses PostgreSQL's ```UNNEST``` function to convert entries in the outgoing adjacency table to an intermediate result that enables subsequent CTEs or queries to process matching edges. Since SQLite doesn't have an equivalent to ```UNNEST```, instead we opt to use column pairs, one with the edge label and the other containing an array of JSON objects, each of which contains the edge and target vertex ids for a given edge that shares the edge label.

In [6]:
def generate_adjacency_tables(db):
    out_neighborhoods = {}
    in_neighborhoods = {}
    for row in db.execute("select sid, eid, tid, label from edge").fetchall():
        (sid, eid, tid, label) = row
        label_column, edges_column = column_pair(label)

        if sid not in out_neighborhoods:
            out_neighborhoods[sid] = { 'vid': sid }
        if label_column not in out_neighborhoods[sid]:
            out_neighborhoods[sid][label_column] = label
        if edges_column not in out_neighborhoods[sid]:
            out_neighborhoods[sid][edges_column] = []
        out_neighborhoods[sid][edges_column].append({ 'eid': eid, 'tid': tid })
        
        if tid not in in_neighborhoods:
            in_neighborhoods[tid] = { 'vid': tid }
        if label_column not in in_neighborhoods[tid]:
            in_neighborhoods[tid][label_column] = label
        if edges_column not in in_neighborhoods[tid]:
            in_neighborhoods[tid][edges_column] = []
        in_neighborhoods[tid][edges_column].append({ 'eid': eid, 'sid': sid })       
        
    db['outgoing'].insert_all([ out_neighborhoods[sid] for sid in out_neighborhoods ], pk='vid')
    db['incoming'].insert_all([ in_neighborhoods[tid] for tid in in_neighborhoods ], pk='vid')

### A utility function for displaying result sets as Pandas DataFrames

Just a little thing to display results in a more readable fashion than simply printing rows out.

In [7]:
def as_dataframe(rows, columns=None):
    df = pd.DataFrame( rows, columns=columns )
    df.index = [''] * len(df)
    return df

## Loading the graph

We hew closely to the example in Schmid's paper, specifically the data shown in Tables 2, 3, and 4 of [[1]](https://dl.acm.org/doi/abs/10.1145/3366030.3366046).

### The vertex table

In [8]:
db['vertex'].insert_all([
    { "vid": 1, "attributes": { "lastName": "Mueller", "firstName": "David" } },
    { "vid": 2, "attributes": { "lastName": "Choi", "firstName": "Jae-Jin" } },
    { "vid": 3, "attributes": { "lastName": "Yamamoto", "firstName": "Akira" } },
    { "vid": 4, "attributes": { "lastName": "Silva", "firstName": "Ana" } },
    { "vid": 5, "attributes": { "lastName": "Poussin", "firstName": "Jacques" } },
    { "vid": 6, "attributes": { "lastName": "Professorson", "firstName": "Derek" } },
    { "vid": 7, "attributes": { "lastName": "Abadi", "firstName": "Madiha" } },
], pk='vid')

<Table vertex (vid, attributes)>

In [9]:
print(db['vertex'].schema)

CREATE TABLE [vertex] (
   [vid] INTEGER PRIMARY KEY,
   [attributes] TEXT
)


In [10]:
as_dataframe( [ row for row in db['vertex'].rows ] )

,vid,attributes
,1,"{""lastName"": ""Mueller"", ""firstName"": ""David""}"
,2,"{""lastName"": ""Choi"", ""firstName"": ""Jae-Jin""}"
,3,"{""lastName"": ""Yamamoto"", ""firstName"": ""Akira""}"
,4,"{""lastName"": ""Silva"", ""firstName"": ""Ana""}"
,5,"{""lastName"": ""Poussin"", ""firstName"": ""Jacques""}"
,6,"{""lastName"": ""Professorson"", ""firstName"": ""Der..."
,7,"{""lastName"": ""Abadi"", ""firstName"": ""Madiha""}"


### The edge table

In [11]:
db['edge'].insert_all([ 
    { "eid": 4, "sid": 1, "tid": 2, "label": "knows", "attributes": { "since": "2018-06-14" } },
    { "eid": 5, "sid": 1, "tid": 3, "label": "knows", "attributes": { "since": "2016-03-21" } },
    { "eid": 6, "sid": 2, "tid": 7, "label": "likes" },
], pk='eid')

<Table edge (eid, sid, tid, label, attributes)>

In [12]:
print(db['edge'].schema)

CREATE TABLE [edge] (
   [eid] INTEGER PRIMARY KEY,
   [sid] INTEGER,
   [tid] INTEGER,
   [label] TEXT,
   [attributes] TEXT
)


In [13]:
as_dataframe( [ row for row in db['edge'].rows ] )

,eid,sid,tid,label,attributes
,4,1,2,knows,"{""since"": ""2018-06-14""}"
,5,1,3,knows,"{""since"": ""2016-03-21""}"
,6,2,7,likes,None


### The adjacency tables

In [14]:
generate_adjacency_tables(db)

In [15]:
print(db['outgoing'].schema)

CREATE TABLE [outgoing] (
   [vid] INTEGER PRIMARY KEY,
   [label_54] TEXT,
   [edges_54] TEXT,
   [label_38] TEXT,
   [edges_38] TEXT
)


In [16]:
as_dataframe( [ row for row in db['outgoing'].rows ] )

,vid,label_54,edges_54,label_38,edges_38
,1,knows,"[{""eid"": 4, ""tid"": 2}, {""eid"": 5, ""tid"": 3}]",None,None
,2,None,None,likes,"[{""eid"": 6, ""tid"": 7}]"


In [17]:
print(db['incoming'].schema)

CREATE TABLE [incoming] (
   [vid] INTEGER PRIMARY KEY,
   [label_54] TEXT,
   [edges_54] TEXT,
   [label_38] TEXT,
   [edges_38] TEXT
)


In [18]:
as_dataframe( [ row for row in db['incoming'].rows ] )

,vid,label_54,edges_54,label_38,edges_38
,2,knows,"[{""eid"": 4, ""sid"": 1}]",None,None
,3,knows,"[{""eid"": 5, ""sid"": 1}]",None,None
,7,None,None,likes,"[{""eid"": 6, ""sid"": 2}]"


## Querying the graph

### A utility function for generating a CTE for edges with targets in the out neighborhood of a given vertex and edge label

Here we define generate a Common Table Expression (CTE) corresponding to the combination of ```unshred_edges``` and ```gather_edges``` in Schmid's article, with the source vertex id and edge label as arguments.

In [19]:
def out_neighborhood_cte(vid, label):
    label_column, edges_column = column_pair(label)
    return (
        f'select vid, json_extract(value, "$.eid") as eid, {label_column} as label,'
        ' json_extract(value, "$.tid") as tid' 
        f' from outgoing, json_each(outgoing.{edges_column})'
        f' where vid = {vid} and {label_column} = "{label}"'
    )

In [20]:
as_dataframe([ row for row in db.execute(out_neighborhood_cte(1, "knows")).fetchall() ],
             columns=['vid', 'eid', 'label', 'tid'] )

,vid,eid,label,tid
,1,4,knows,2
,1,5,knows,3


In [21]:
as_dataframe([ row for row in db.execute(out_neighborhood_cte(2, "likes")).fetchall() ],
             columns=['vid', 'eid', 'label', 'tid'] )

,vid,eid,label,tid
,2,6,likes,7


### Who does David Mueller know?

In [22]:
query_1 = (
    f'with unshred_edges as ( {out_neighborhood_cte(1, "knows")} ),'
    ' targets as ( select tid from unshred_edges )'
    ' select json_extract(attributes, "$.lastName"),'
    ' json_extract(attributes, "$.firstName")'
    ' from vertex, targets where vertex.vid = tid'
)

In [23]:
as_dataframe( [ row for row in db.execute(query_1).fetchall() ],
             columns=['lastName', 'firstName'] )

,lastName,firstName
,Choi,Jae-Jin
,Yamamoto,Akira


### Who do people David Mueller knows like? 

#### Using the edge table

In [24]:
query_2 = (
    f'select json_extract(vertex.attributes, "$.lastName"),'
    ' json_extract(vertex.attributes, "$.firstName")'
    ' from vertex, edge as e1, edge as e2 where'
    ' e1.sid = 1 and'
    ' e1.label = "knows" and'
    ' e2.sid = e1.tid and'
    ' e2.label = "likes" and'
    ' vertex.vid = e2.tid'
)

In [25]:
as_dataframe( [ row for row in db.execute(query_2).fetchall() ],
             columns=['lastName', 'firstName'] )

,lastName,firstName
,Abadi,Madiha


#### Using the outgoing (adjacency) table

In [28]:
query_3 = (
    f'with knows_edges as ( {out_neighborhood_cte(1, "knows")} ),'
    ' likes_edges as (select outgoing.vid as vid, json_extract(value, "$.eid") as eid, label_38 as label, json_extract(value, "$.tid") as tid from knows_edges, outgoing, json_each(outgoing.edges_38) where outgoing.vid = knows_edges.tid and label_38 = "likes"),'
    ' targets as ( select tid from likes_edges )'
    ' select json_extract(attributes, "$.lastName"),'
    ' json_extract(attributes, "$.firstName")'
    ' from vertex, targets where vertex.vid = tid'
)

In [29]:
as_dataframe( [ row for row in db.execute(query_3).fetchall() ],
             columns=['lastName', 'firstName'] )

,lastName,firstName
,Abadi,Madiha


### Who has David Mueller known the longest?

In [30]:
query_4 = (
    f'with unshred_edges as ( {out_neighborhood_cte(1, "knows")} ),'
    ' targets as ( select eid, tid from unshred_edges )'
    ' select json_extract(vertex.attributes, "$.lastName"),'
    ' json_extract(vertex.attributes, "$.firstName"),'
    ' min(json_extract(edge.attributes, "$.since"))'
    ' from vertex, edge, targets where vertex.vid = targets.tid and edge.eid = targets.eid'
)

In [31]:
as_dataframe( [ row for row in db.execute(query_4).fetchall() ],
             columns=['lastName', 'firstName', 'since'] )

,lastName,firstName,since
,Yamamoto,Akira,2016-03-21


### How many "knows" edges are there?

In [32]:
query_5 = f'select count(eid) from edge where label = "knows"'

In [33]:
db.execute(query_5).fetchone()[0]

2

### How many "knows" edges were created since 2017?

In [34]:
query_6 = (
    f'select count(eid) '
    'from edge where label = "knows" and '
    'json_extract(edge.attributes, "$.since") > "2017"'
)

In [35]:
db.execute(query_6).fetchone()[0]

1

### How many vertices have last names that start with the letter "P"?

In [36]:
query_7 = f'select count(vid) from vertex where json_extract(vertex.attributes, "$.lastName") LIKE "P%"'

In [37]:
db.execute(query_7).fetchone()[0]

2

## References

[[1]](https://dl.acm.org/doi/abs/10.1145/3366030.3366046) Schmid, M., 2019, December. On efficiently storing huge property graphs in relational database management systems. In Proceedings of the 21st International Conference on Information Integration and Web-based Applications & Services (pp. 344-352).

[[2]](https://d1wqtxts1xzle7.cloudfront.net/69945800/Building_an_efficient_RDF_store_over_a_r20210919-15810-1bebrk0-with-cover-page-v2.pdf?Expires=1652064544&Signature=DoStFyoFgEEBo5sRj1nCLaclUvoTXfhywyMcoytJNboAy~1Zf0LQi~SAeS1guJXpsi4QfS2F7Pbso3rK~U-VNPzJxMgY1mC0IgjoqcuXh025cmvKJJi-OTTb1S87tbBaEi0Dd20bf39-jaLdI1u97iDOdj88uUcaOC14m54DgxdMm9nc1FpZzM-DuyP-LWD-eyQ0qoNxZqjnCs48Tjb9RjAy3OgaAdXlEgLukQWJezhVdbkjzuG8Qh~egRpXzAAJurN4pUYk7F6UC~8Ep5IPm0Z1qO~kDCYuJ7MIAGuE-8Z9M1zb-BI3BHF7tgphspndjVhQQLX2R5JW4rjtQvNZfQ__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA) Bornea, M.A., Dolby, J., Kementsietsidis, A., Srinivas, K., Dantressangle, P., Udrea, O. and Bhattacharjee, B., 2013, June. Building an efficient RDF store over a relational database. In Proceedings of the 2013 ACM SIGMOD International Conference on Management of Data (pp. 121-132).

[[3]](https://dl.acm.org/doi/abs/10.1145/2723372.2723732) Sun, W., Fokoue, A., Srinivas, K., Kementsietsidis, A., Hu, G. and Xie, G., 2015, May. Sqlgraph: An efficient relational-based property graph store. In Proceedings of the 2015 ACM SIGMOD International Conference on Management of Data (pp. 1887-1901).